## Luca Analysis

Which store generates the most revenue?

In [8]:

# Import libraries and connection to Util
import pandas as pd
import sys 
sys.path.append('..')
from src.utils.db_conn import db_conn
from src.utils.db_engine import db_engine

# Which store generated the most Rev.

In [10]:
# Create SQL engine
engine = db_engine()

# SQL query with schema-qualified tables
query = """
SELECT s.store_id, SUM(p.amount) AS total_revenue
FROM main.payment p
JOIN main.staff s ON p.staff_id = s.staff_id
JOIN main.store st ON s.store_id = st.store_id
GROUP BY s.store_id
ORDER BY total_revenue DESC;
"""

# Execute query
revenue_by_store = pd.read_sql(query, engine)
revenue_by_store.head(20)


,store_id,total_revenue
0,2,31059.92
1,1,30252.12


# Which staff member took the most payments with total rev and avg payment

In [16]:
# SQL queries 
payments_query = "SELECT * FROM main.payment;"
staff_query = "SELECT * FROM main.staff"
# Load into pd df
payments = pd.read_sql(payments_query, engine)
staff = pd.read_sql(staff_query, engine)



staff_perf = payments.merge(staff, on="staff_id")

staff_summary = (
    staff_perf.groupby(["staff_id", "first_name", "last_name"])
    .agg(
        total_payments=("payment_id", "count"),
        total_revenue=("amount", "sum"),
        avg_payment=("amount", "mean")
    )
    .sort_values("total_revenue", ascending=False)
)

# staff_summary.plot(kind="bar", y="total_revenue", title="Staff Revenue Leaderboard")

staff_summary.head(5)




,,,total_payments,total_revenue,avg_payment
staff_id,first_name,last_name,,,
2,Jon,Stephens,7304,31059.92,4.252453
1,Mike,Hillyer,7292,30252.12,4.148673


# Which staff member made the most rentals 

In [17]:

rentals_query = "SELECT * FROM main.rental;"
staff_query = "SELECT * FROM main.staff;"

rentals = pd.read_sql(rentals_query, engine)
staff = pd.read_sql(staff_query, engine)

staff_rentals = rentals.merge(staff, on="staff_id")

rental_counts = (
    staff_rentals.groupby(["staff_id", "first_name", "last_name"])
    .agg(total_rentals=("rental_id", "count"))
    .sort_values("total_rentals", ascending=False)
)

rental_counts.head(5)

,,,total_rentals
staff_id,first_name,last_name,
1,Mike,Hillyer,8040
2,Jon,Stephens,8004


# Top 5 Expensive Movie Categories by average price of rental

In [20]:
engine = db_engine()

query = '''
SELECT 
    c.name AS category,
    AVG(p.amount) AS avg_price,
    SUM(p.amount) AS total_revenue,
    COUNT(p.payment_id) AS num_payments
FROM 
    main.payment p
JOIN 
    main.rental r ON p.rental_id = r.rental_id
JOIN 
    main.inventory i ON r.inventory_id = i.inventory_id
JOIN 
    main.film f ON i.film_id = f.film_id
JOIN 
    main.film_category fc ON f.film_id = fc.film_id
JOIN 
    main.category c ON fc.category_id = c.category_id
GROUP BY 
    c.name
ORDER BY 
    avg_price DESC
LIMIT 10;
'''

top_3_categories_by_avg = pd.read_sql(query, engine)
top_3_categories_by_avg.head()


,category,avg_price,total_revenue,num_payments
0,Comedy,4.703267,4002.48,851
1,New,4.590718,3966.38,864
2,Sports,4.525615,4892.19,1081
3,Games,4.436855,3922.18,884
4,Horror,4.400091,3401.27,773


# bottom 5 Expensive Movie Categories by average price of rental

In [21]:
engine = db_engine()

query = '''
SELECT 
    c.name AS category,
    AVG(p.amount) AS avg_price,
    SUM(p.amount) AS total_revenue,
    COUNT(p.payment_id) AS num_payments
FROM 
    main.payment p
JOIN 
    main.rental r ON p.rental_id = r.rental_id
JOIN 
    main.inventory i ON r.inventory_id = i.inventory_id
JOIN 
    main.film f ON i.film_id = f.film_id
JOIN 
    main.film_category fc ON f.film_id = fc.film_id
JOIN 
    main.category c ON fc.category_id = c.category_id
GROUP BY 
    c.name
ORDER BY 
    avg_price ASC
LIMIT 10;
'''

top_3_categories_by_avg = pd.read_sql(query, engine)
top_3_categories_by_avg.head()

,category,avg_price,total_revenue,num_payments
0,Children,3.843659,3309.39,861
1,Family,3.876670,3830.15,988
2,Classics,3.899279,3353.38,860
3,Action,3.901125,3951.84,1013
4,Animation,3.986207,4245.31,1065
